In [8]:
import asyncio
import aiohttp
from datetime import datetime, timedelta

In [9]:
async def get_advert_spend(account: str, token: str, date_from: str, date_to: str, api_token: str, session: aiohttp.ClientSession):
    """Получает данные по рекламным затратам за указанный период"""
    url = "https://advert-api.wildberries.ru/adv/v1/upd"
    headers = {
        "Authorization": api_token
    }
    params = {
        "from": date_from,
        "to": date_to
        }
    # Интервал для запросов на ВБ
    delay = 1
    # Количество попыток в случае ошибки
    attempts = 3
    for attempt in range(attempts+1):
        try:
                async with session.get(url, headers=headers, params=params, timeout=10) as res:
                    # 1. Пытаемся распарсить JSON, если это возможно
                    content_type = res.headers.get('Content-Type', '')
                    data = await res.json() if 'application/json' in content_type else None

                    if res.status == 200:
                        for d in data:
                            d['account'] = account.upper() # Добавляем информацию об аккаунте в данные
                        print(f"✅ [{account}] Данные получены за {date_from}")
                        return data             

                    # 2. Безопасно достаем описание ошибки
                    error_detail = data.get('detail') if data else await res.text()
                    
                    # 3. Обработка ошибок без дублирования кода
                    if res.status == 401:
                        print(f"🔑 [{account}] Ошибка 401: Неверный токен. ({error_detail})")
                    elif res.status == 429:
                        print(f"⏳ [{account}] Ошибка 429: Лимит запросов! ({error_detail})")
                        asyncio.sleep(delay)
                        continue
                    elif res.status == 400:
                        print(f"❓ [{account}] Ошибка 400: Плохой запрос. ({error_detail})")
                    else:
                        print(f"❌ [{account}] Ошибка {res.status}: {error_detail}")
                    
                    return None
                    
        except Exception as e:
                print(f"💥 [{account}] Непредвиденная ошибка: {e}")
                return None


async def fetch_advert_spend_info(tokens: dict, date_from, date_to):
    """Асинхронная функция для получения информации о рекламных кампаниях для всех аккаунтов и токенов."""
    # Асинхронно обрабатываем все аккаунты и токены
    async with aiohttp.ClientSession() as session:
            # Создаем задачи для каждого аккаунта и токена
            tasks = [get_advert_spend(account, token, date_from, date_to, token, session) for account, token in tokens.items()]
            # Ожидаем завершения всех задач и собираем результаты
            results = await asyncio.gather(*tasks)
            return results   

In [10]:
# Импортируем библиотеки для работы с системой и путями
import os
from  pathlib import Path
import sys
# Установка базовой директории и пути к файлу с учетными данными. Используем конструкцию try-except для обработки возможных ошибок при определении пути для notebook.
try:
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    BASE_DIR = Path.cwd().resolve().parents[1] 

SRC_DIR = os.path.join(BASE_DIR, 'src')

# Добавляем src в пути поиска модулей
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Импорт самописных утилит
from core.utils_general import load_api_tokens


In [ ]:
import pandas as pd

def process_advert_spend_info(data, date_from):
    """ Функция обрабатывает полученные данные по рекламным затратам за один день. Метод АПИ возвращает лишние даты, фунцкия так же производит фильтрацию"""
    if data:
        # Список для хранения данных
        data_list = []
        # Проходим внешним циклом по данным каждого ЛК
        for orders_lk in data:
            # Достаем отдельно данные по каждому заказу и добваляем в общий список
            for orders in orders_lk:
                data_list.append(orders)
        df = pd.DataFrame(data_list)
        df['date'] = pd.to_datetime(df['updTime'],
                                      format='ISO8601',
                                      utc=True).dt.date
        df = df.loc[df['date'] == pd.to_datetime(date_from).date()]
        return df
    else:
        return pd.DataFrame() 
    

async def advert_spend_info(days_count=28):
    """ Обрабатывает данные собранные за несколько дней. По умолчанию за 28 дней."""
    # Список для хранения данных
    all_data = []
    # Проходим циклом по каждому дню
    for day in range(1, days_count+1):
        date_from = date_to = (datetime.now()-timedelta(days=day)).strftime("%Y-%m-%d")
        data = await fetch_advert_spend_info(load_api_tokens(), date_from, date_to)
        try:
            df_data = process_advert_spend_info(data, date_from)   
            all_data.append(df_data)
        except Exception:
            all_data.append(pd.DataFrame())

    df_all_data = pd.concat(all_data)
    return df_all_data

✅ [ВЕКТОР3522] Данные получены за 2026-02-07
✅ [СТАРТ9237] Данные получены за 2026-02-07
✅ [СТАРТ4040] Данные получены за 2026-02-07
✅ [ВЕКТОР4216] Данные получены за 2026-02-07
✅ [Старт2] Данные получены за 2026-02-07
✅ [Вектор2] Данные получены за 2026-02-07
✅ [ВЕКТОР7794] Данные получены за 2026-02-07
✅ [СТАРТ8748] Данные получены за 2026-02-07
✅ [Лопатина] Данные получены за 2026-02-07
✅ [Даниелян] Данные получены за 2026-02-07
✅ [Хачатрян] Данные получены за 2026-02-07
✅ [Оганесян] Данные получены за 2026-02-07
✅ [Старт] Данные получены за 2026-02-07
✅ [Пилосян] Данные получены за 2026-02-07
✅ [Тоноян] Данные получены за 2026-02-07
✅ [Вектор] Данные получены за 2026-02-07
✅ [ВЕКТОР3522] Данные получены за 2026-02-06
✅ [СТАРТ8748] Данные получены за 2026-02-06
✅ [Старт2] Данные получены за 2026-02-06
✅ [СТАРТ9237] Данные получены за 2026-02-06
✅ [Вектор2] Данные получены за 2026-02-06
✅ [Даниелян] Данные получены за 2026-02-06
✅ [Хачатрян] Данные получены за 2026-02-06
✅ [СТАРТ4040

In [ ]:
from sqlalchemy import BigInteger, String, Boolean, DateTime
from core.utils_sql import get_db_engine, sync_data_to_postgres
from modules.WB.advert.schemas import advert_spend_info_dict

def advert_spend():
    """Получение данных по затратам"""
    df = asyncio.run(advert_spend_info())
    # Присваиваем имена колонок в snake_case
    df = df.rename(columns={
        'updTime': 'upd_time',
        'campName': 'camp_name',
        'paymentType': 'payment_type',
        'updNum': 'upd_num',
        'updSum': 'upd_sum',
        'advertId': 'advert_id',
        'advertType': 'advert_type',
        'advertStatus': 'advert_status'
                            })
    # Преобразуем датафрейм в словарь для вставкив БД
    data = df.to_dict(orient='records')
    # Установка соединения с базой данных
    engine = get_db_engine()

    table_name = list(advert_spend_info_dict.keys())[0]
    unique_keys = ['advert_id', 'upd_num', 'upd_time']  # Уникальность по ID кампании
    # В ключе указываем имя таблицы в БД
    schema_definition = advert_spend_info_dict.get(table_name)

    sync_data_to_postgres(engine, table_name, data, schema_definition, unique_keys)